In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np

D:\sasnjeev\anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
input_df = pd.read_csv(".\\data\\train.csv")

In [3]:
input_df.head(5)

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
input_df['Browser_Used'] = le.fit_transform(input_df['Browser_Used'])
le.classes_

array(['Chrome', 'Edge', 'Firefox', 'Google Chrome', 'IE',
       'Internet Explorer', 'InternetExplorer', 'Mozilla',
       'Mozilla Firefox', 'Opera', 'Safari'], dtype=object)

In [5]:
le2 = LabelEncoder()
input_df['Device_Used'] = le2.fit_transform(input_df['Device_Used'])
# input_df.head()
le2.classes_

array(['Desktop', 'Mobile', 'Tablet'], dtype=object)

In [6]:
# input_df['stemmed_desc'] = stemmer.stem(input_df.describe)
from sklearn.model_selection import train_test_split
train,test = train_test_split(input_df, test_size=0.33, random_state=42)

In [7]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")


In [11]:
print train["Description"][1]
print "88888888888888888"
example_words = ["python","pythoner","pythoning","pythoned","pythonly"]
for i in example_words:
    print stemmer.stem(i)

I stayed at the Crown Plaza April -- - April --, ----. The staff was friendly and attentive. The elevators are tiny (about -' by -'). The food in the restaurant was delicious but priced a little on the high side. Of course this is Washington DC. There is no pool and little for children to do. My room on the fifth floor had two comfortable beds and plenty of space for one person. The TV is a little small by todays standards with a limited number of channels. There was a small bit of mold in the bathtub area that could have been removed with a little bleach. It appeared the carpets were not vacummed every day. I reported a light bulb was burned out. It was never replaced. Ice machines are on the odd numbered floors, but the one on my floor did not work. I encountered some staff in the elevator one evening and I mentioned the ice machine to them. Severel hours later a maid appeared at my door with ice and two mints. I'm not sure how they knew what room I was in. That was a little unnervin

In [213]:
print train.shape
print test.shape

(26084, 5)
(12848, 5)


In [214]:
y_train = train.Is_Response
y_test = test.Is_Response
X_train = train.drop('Is_Response', axis=1)
X_test = test.drop('Is_Response', axis=1)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [215]:
print X_train.shape
print X_train.columns
print X_test.shape
print X_test.columns
print y_train.shape
# print y_train.columns
print y_test.shape
# print y_test.columns

(26084, 4)
Index([u'User_ID', u'Description', u'Browser_Used', u'Device_Used'], dtype='object')
(12848, 4)
Index([u'User_ID', u'Description', u'Browser_Used', u'Device_Used'], dtype='object')
(26084, 1)
(12848, 1)


In [216]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cv = CountVectorizer(stop_words='english', min_df=5, ngram_range=(1,2), max_features=500)
cv.fit(X_train.Description.values.tolist()+X_test.Description.values.tolist())
tfidf = TfidfVectorizer(stop_words='english', min_df=5, ngram_range=(1,2), max_features=500)
tfidf.fit(X_train.Description.values.tolist()+X_test.Description.values.tolist())


TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=500, min_df=5,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [217]:
# occ = np.asarray(cv_count.sum(axis=0)).ravel().tolist()
# counts_df = pd.DataFrame({'term': cv.get_feature_names(), 'occurrences': occ})
# counts_df.sort_values(by='occurrences', ascending=False).head(20)
train_desc_vec = cv.transform(X_train.Description.values.tolist())
test_desc_vec = cv.transform(X_test.Description.values.tolist())
train_desc_tfidf = tfidf.transform(X_train.Description.values.tolist())
test_desc_tfidf = tfidf.transform(X_test.Description.values.tolist())

In [218]:
# transformer = TfidfTransformer()
# train_desc_tfidf = transformer.fit_transform(train_desc_vec)
# test_desc_tfidf = transformer.fit_transform(test_desc_vec)

In [219]:
print train_desc_vec.get_shape()
print test_desc_vec.get_shape()
print train_desc_tfidf.get_shape()
print test_desc_tfidf.get_shape()

(26084, 500)
(12848, 500)
(26084, 500)
(12848, 500)


In [220]:
print type(train_desc_vec)
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
print y_train[:5]
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)
print lb.classes_
type(y_train)

<class 'scipy.sparse.csr.csr_matrix'>
      Is_Response
5453        happy
8832    not happy
29812       happy
30177       happy
24178   not happy
['happy' 'not happy']


numpy.ndarray

In [221]:
# X_train[cols].astype(float).values

In [222]:
cols = ['Browser_Used', 'Device_Used']
xtrain = hstack([csr_matrix(X_train[cols].astype(float).values), train_desc_vec, train_desc_tfidf])
xtest = hstack([csr_matrix(X_test[cols].astype(float).values), test_desc_vec, test_desc_tfidf])

In [223]:
dtrain = xgb.DMatrix(xtrain, label=y_train)
dtest = xgb.DMatrix(xtest, label=y_test)

In [224]:
type(y_train)

numpy.ndarray

In [225]:
params = {'objective': 'binary:logistic',
          'booster': 'gbtree',
          'eval_metric': 'error',
          'nthread': 4,
          'silent': 1,
          'max_depth': 6,
          'subsample': 0.9,
          'min_child_weight': 1,
          "colsample_bytree": 0.9,
          'eta': 0.1,
          'seed': 2017}
watchlist = [(dtrain, 'train'), (dtest, 'test')]
num_rounds = 200
clf_xgb = xgb.train(params, dtrain, num_rounds, verbose_eval=10, evals=watchlist, early_stopping_rounds=10)

[0]	train-error:0.211049	test-error:0.226806
Multiple eval metrics have been passed: 'test-error' will be used for early stopping.

Will train until test-error hasn't improved in 10 rounds.
[10]	train-error:0.181107	test-error:0.200342
[20]	train-error:0.162782	test-error:0.185554
[30]	train-error:0.150054	test-error:0.175747
[40]	train-error:0.137134	test-error:0.16773
[50]	train-error:0.125863	test-error:0.161582
[60]	train-error:0.11739	test-error:0.157534
[70]	train-error:0.110489	test-error:0.153876
[80]	train-error:0.104854	test-error:0.149362
[90]	train-error:0.099601	test-error:0.145781
[100]	train-error:0.094732	test-error:0.143602
[110]	train-error:0.0904	test-error:0.143836
[120]	train-error:0.087563	test-error:0.141501
[130]	train-error:0.083193	test-error:0.140333
Stopping. Best iteration:
[125]	train-error:0.085033	test-error:0.13971



In [226]:
pred = clf_xgb.predict(dtest)

In [227]:
print pred[:5]
print y_test[:5]
pred_ = pred>0.5

[ 0.09318551  0.12945217  0.0466382   0.03097434  0.97867715]
[[0]
 [0]
 [0]
 [0]
 [1]]


In [228]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, pred_)
print score

0.859355541719


In [229]:
submit_df = pd.read_csv(".\\data\\test.csv")
submit_label = submit_df["User_ID"]
submit_df.drop("User_ID", axis=1, inplace=True)
submit_df['Browser_Used'] = le.transform(submit_df['Browser_Used'])
submit_df['Device_Used'] = le2.transform(submit_df['Device_Used'])

In [230]:
cols = ['Browser_Used', 'Device_Used']
submit_desc_vec = cv.transform(submit_df.Description.values.tolist())
submit_desc_tfidf = tfidf.transform(submit_df.Description.values.tolist())
submit_test = hstack([csr_matrix(submit_df[cols].astype(float).values), submit_desc_vec, submit_desc_tfidf])
# submit_df.head(5)

In [231]:
dsubmittest = xgb.DMatrix(submit_test)

In [232]:
print submit_pred.shape
print submit_pred[:5]

(29404, 1)
           0
0  not happy
1      happy
2      happy
3  not happy
4      happy


In [233]:
submit_pred = clf_xgb.predict(dsubmittest)

In [234]:
print submit_pred[:5]
submit_pred = pd.DataFrame(submit_pred)
submit_pred[0].value_counts()
submit_pred = submit_pred>0.5
print submit_pred[0].value_counts()
submit_pred.replace({True:'not happy',False:'happy'}, inplace=True)
print submit_pred[0].value_counts()
# submit_label = pd.concat([submit_label, pd.DataFrame(sub)])

[ 0.8542859   0.04171207  0.22390249  0.59733105  0.00784046]
False    21340
True      8064
Name: 0, dtype: int64
happy        21340
not happy     8064
Name: 0, dtype: int64


In [235]:
print submit_label.shape
submit_label = pd.concat([submit_label,submit_pred], axis=1)
print submit_label.shape

(29404L,)
(29404, 2)


In [236]:

submit_label.columns = ["User_ID","Is_Response"]
submit_label.to_csv("submission.csv", index=False)

In [237]:
df = pd.read_csv(".\\data\\train.csv")
print df["Is_Response"].value_counts()
e = LabelBinarizer()
t = e.fit_transform(df["Is_Response"])
t = pd.DataFrame(t)
print pd.DataFrame(t)[0].value_counts()
t = e.inverse_transform(t)

print pd.DataFrame(t)[0].value_counts()

happy        26521
not happy    12411
Name: Is_Response, dtype: int64
0    26521
1    12411
Name: 0, dtype: int64
happy        26521
not happy    12411
Name: 0, dtype: int64
